In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string
import contractions
from num2words import num2words
from gensim.models import Word2Vec
import re
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

dataSet = pd.read_excel('/Users/amulya/Downloads/text_columns.xlsx')
dataSet.info()
# 1900


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1899 entries, 0 to 1898
Data columns (total 19 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Company                     1899 non-null   object 
 1   Lists                       1496 non-null   object 
 2   Employees                   1713 non-null   float64
 3   Industry                    1827 non-null   object 
 4   Company City                1835 non-null   object 
 5   Company State               1864 non-null   object 
 6   Company Country             1891 non-null   object 
 7   Keywords                    1630 non-null   object 
 8   SEO Description             1250 non-null   object 
 9   Technologies                1847 non-null   object 
 10  Total Funding               768 non-null    float64
 11  Latest Funding              848 non-null    object 
 12  Latest Funding Amount       669 non-null    object 
 13  Last Raised At              848 n

In [2]:
dataSet['ICP'].value_counts()

dataSet['ICP'] = dataSet['ICP'].replace({'ICP - 2': 1, 'ICP-1': 1, 'ICP - 1': 1, 'Not an ICP' : 0})
dataSet['ICP'].value_counts()

ICP
1    1010
0     889
Name: count, dtype: int64

In [3]:
dataSet['SEO Description']

0       Gainsight's Customer Success and Product Exper...
1                                                     NaN
2       Build production-ready web & mobile apps faste...
3       Runo Call Management App to manage calls at SI...
4       Accelerate your eCommerce data and analytics j...
5                                                     NaN
6       Boost your business with OneHash, a powerful a...
7       Motadata is a comprehensive IT monitoring and ...
8       Zithara helps brands build omichannel rewards ...
9                                                     NaN
10                                                    NaN
11      FreJun automates calling, logging your busines...
12      Globally trusted Conversational AI Platform fo...
13      Prismforce offers AI/ML based vertical SaaS fo...
14      Automate your business & simplify your life wi...
15                                                    NaN
16                                                    NaN
17      Rattle

In [4]:
def selectColumns(df):
    requiredColumns = ['Company', 'Lists', 'Industry', 'Keywords', 'SEO Description',
       'Technologies', 'Short Description', 'ICP']

    return df[requiredColumns]


In [5]:
dataFrame = selectColumns(dataSet)
dataFrame['is_ICP'] = dataFrame['ICP']

In [6]:
from sklearn.model_selection import train_test_split

x = dataFrame.drop(['is_ICP', 'ICP'],axis = 1)
y = dataFrame['is_ICP']
print(y.value_counts())
xTrain, xTest, yTrain, yTest = train_test_split(x, y, test_size=0.2, random_state=42)

trainBkp = xTrain.copy()
testBkp = xTest.copy()
print(yTest)

is_ICP
1    1010
0     889
Name: count, dtype: int64
1222    1
111     1
554     1
526     1
1310    0
1100    0
70      1
251     1
1314    0
1029    0
1731    0
1562    0
324     0
128     1
196     0
1831    1
1047    1
1287    0
438     1
1324    1
1271    1
331     1
1670    1
532     1
1853    1
471     0
135     1
1897    1
1362    0
1061    1
1235    1
1765    1
1651    0
1368    0
1856    1
1216    0
1113    0
985     0
530     0
256     1
940     1
916     1
544     1
1211    0
1574    0
1164    0
411     1
1206    0
99      0
1469    1
1223    1
637     1
988     0
1063    0
552     1
1079    1
519     1
1022    0
218     1
1355    0
1594    0
1106    0
382     1
1319    0
203     0
706     0
1776    0
678     1
1175    1
1596    0
1473    0
1521    1
237     0
65      0
1827    0
188     0
721     1
1890    0
29      1
1244    0
781     1
432     1
275     0
1489    0
231     0
1813    0
859     0
1788    1
518     1
482     1
654     0
925     1
1754    1
1487    0
1637   

In [7]:
xTrain.reset_index(drop=True, inplace=True)
xTest.reset_index(drop=True, inplace=True)
yTrain.reset_index(drop=True, inplace=True)
yTest.reset_index(drop=True, inplace=True)

In [8]:
def convStrToList(df, column):
    df[column] = df[column].fillna('')
    df[column] = df[column].str.split(',')
    df[column] = df[column].fillna('')
    df[column] = [[col.strip() for col in sublist] for sublist in df[column]]
    
    allColumns = [col.strip() for sublist in df[column] for col in sublist]
    allColumns = set([col for col in allColumns if col != ''])
    allColumns = sorted(allColumns)
    
    return df[column]

In [9]:
def preProcessText(df, column):
    df[column] = df[column].astype(str)
    df[column] = df[column].str.lower()
    
    df[column] = df[column].apply(word_tokenize)
    punctuation = set(string.punctuation)
    df[column] = df[column].apply(lambda tokens: [token for token in tokens if token not in punctuation])

    stop_words = set(stopwords.words('english'))
    df[column] = df[column].apply(lambda tokens: [token for token in tokens if token not in stop_words])

    lemmatizer = WordNetLemmatizer()
    df[column] = df[column].apply(lambda tokens: [lemmatizer.lemmatize(token) for token in tokens])
    df[column] = df[column].apply(lambda tokens: [num2words(token) if token.isdigit() else token for token in tokens])
    df[column] = df[column].apply(lambda tokens: [contractions.fix(token) for token in tokens])
    df[column] = df[column].apply(lambda tokens: [token for token in tokens if token.isalnum()])
    df[column] = df[column].apply(lambda tokens: ' '.join(tokens))

    return df[column]

In [10]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MultiLabelBinarizer
import pickle

def genIndustryLabels(df):
    indLabels = LabelBinarizer()
    df['Industry'] = df['Industry'].fillna('')
    indLabels.fit(df['Industry'])
    with open('Tam2_industry.pkl', 'wb') as f:
        pickle.dump(indLabels, f)

def genTechnologyLabels(df):
    techLabels = MultiLabelBinarizer()
    techLabels.fit(df['Technologies'])
    with open('Tam2_technology.pkl', 'wb') as f:
        pickle.dump(techLabels, f)

def genKeywordLabels(df):
    keywordLabels = MultiLabelBinarizer()
    keywordLabels.fit(df['Keywords'])
    with open('Tam2_keyword.pkl', 'wb') as f:
        pickle.dump(keywordLabels, f)

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

def getVectorsOfText(df):
    tfidfVectorizer = TfidfVectorizer()
    tfidfMatrix = tfidfVectorizer.fit(df['SEO Description'])
    with open('Tam2_seoDes.pkl', 'wb') as f:
        pickle.dump(tfidfMatrix, f)
        
    return tfidfMatrix

In [12]:
def unPickelData(df):
    import pickle
    with open('Tam2_seoDes.pkl', 'rb') as f:
        seoPickeler = pickle.load(f)
        
    seoMatrix = seoPickeler.transform(df['SEO Description'])
    seoColumns = seoPickeler.get_feature_names_out()
    seoDummies = pd.DataFrame(seoMatrix.toarray(), columns = seoColumns)
    df.reset_index(drop=True, inplace=True)
    seoDummies.reset_index(drop=True, inplace=True)

    # Industries
    with open('Tam2_industry.pkl', 'rb') as f:
        industryPickeler = pickle.load(f)
        
    df['Industry'] = df['Industry'].fillna('')
    industryMatrix = industryPickeler.transform(df['Industry'])
    industryColumns = industryPickeler.classes_
    industryDummies = pd.DataFrame(industryMatrix, columns=industryColumns)
    industryDummies.reset_index(drop=True, inplace=True)

    # Technologies
    with open('Tam2_technology.pkl', 'rb') as f:
        techPickler = pickle.load(f)
    
    df['Technologies'] = df['Technologies'].fillna('')
    techMatrix = techPickler.transform(df['Technologies'])
    techColumns = techPickler.classes_
    techDummies = pd.DataFrame(techMatrix, columns = techColumns)
    techDummies.reset_index(drop=True, inplace=True)

   # Keywords
    with open('Tam2_keyword.pkl', 'rb') as f:
        keywordPickler = pickle.load(f)
        
    df['Keywords'] = df['Keywords'].fillna('')
    keywordMatrix = keywordPickler.transform(df['Keywords'])
    keywordColumns = keywordPickler.classes_
    keywordDummies = pd.DataFrame(keywordMatrix, columns = keywordColumns)
    keywordDummies.reset_index(drop=True, inplace=True)
    
    data = pd.concat([ industryDummies, techDummies, keywordDummies, seoDummies], axis=1) 
    # data = pd.concat([ industryDummies, techDummies, keywordDummies], axis=1) 
    # data = pd.concat(df['SEO Description'],axis = 1)
    return data

In [13]:
def preProcessData(df):
    df['Technologies'] = convStrToList(df, 'Technologies')
    df['Keywords'] = convStrToList(df, 'Keywords')
    df['SEO Description'] = preProcessText(df, 'SEO Description')
    return df
    
def pickelTrain(df):
    genIndustryLabels(df)
    genTechnologyLabels(df)
    genKeywordLabels(df)
    getVectorsOfText(df)
    
def dropDuplicateColumns(df):
    duplicateColumns = df.columns[df.columns.duplicated()]
    temp = pd.DataFrame()

    for col in duplicateColumns:
        duplicateIndices = df.columns.get_loc(col)
        temp[col] = df.iloc[:, duplicateIndices].any(axis=1)
    
    df = df.drop(duplicateColumns, axis =1)
    df = pd.concat([df,temp],axis = 1)
    
    return df

In [14]:
def filterTechs(df,data, column):
#     techRes = {}
    unwantedTechs = []
    lowQuantityTechs = []
#     wantedTechs = []
    
    data[column] = data[column].fillna('')
    allTechnologies = [tech.strip() for sublist in data[column] for tech in sublist]
    allTechnologies = set([tech for tech in allTechnologies if tech != ''])
    allTechnologies = sorted(allTechnologies)
    
    for tech in allTechnologies:
        filteredData = df[df[tech] == 1]
        totalRecords = len(filteredData) 
        percentage = filteredData.groupby('is_ICP')['is_ICP'].count() / totalRecords * 100
#         techRes[tech] = {'percentage' : percentage , 'no_of_records' : totalRecords}
        notIcp = percentage.get(0,0)
        icp = percentage.get(1,0)

        if((notIcp > 30) and (notIcp < 70)):
            unwantedTechs.append(tech)
    
        elif (totalRecords < 5):
            lowQuantityTechs.append(tech)
#         else:
#             wantedTechs.append(tech)
    
#     temp = pd.DataFrame()
#     temp[column] = wantedTechs
#     filePath = "Tam2_"+column+".xlsx"
#     temp.to_excel(filePath, index=False)

    return unwantedTechs + lowQuantityTechs


In [15]:

def getTrainX(trainX, trainY):
    
    trainX = preProcessData(trainX)
    trainBkp_1 = trainX.copy()
    pickelTrain(trainX)
    trainX = unPickelData(trainX)
    trainX = dropDuplicateColumns(trainX)
    trainXY = pd.concat([trainX, trainY], axis = 1)
    
    unwantedTechs = filterTechs(trainXY, trainBkp_1, 'Technologies')
    unwantedKeywords = filterTechs(trainXY, trainBkp_1, 'Keywords')
    
    combinedUnwanted = list(set(unwantedTechs + unwantedKeywords))
    with open('Tam2_unwantedCols.pkl', 'wb') as f:
        pickle.dump(combinedUnwanted, f)
        
    trainX = trainX.drop(combinedUnwanted, axis = 1)
    return trainX

In [16]:
from sklearn.linear_model import LogisticRegression

def trainModel(trainX, trainY):
    # Initialize and fit LogisticRegression
    model = LogisticRegression(random_state=5, penalty='l2', max_iter=200)
    print(list(set(trainY)))
    model.fit(trainX, trainY)
    
    with open('Tam2_model.pkl', 'wb') as f:
        pickle.dump(model, f)

def predict(x):
    with open('Tam2_model.pkl', 'rb') as f:
        model = pickle.load(f)
        y = model.predict(x)
        return y
        

In [17]:
def getTestX(xTest):
    testX = preProcessData(xTest)
    testX = unPickelData(testX)
    testX = dropDuplicateColumns(testX)
    with open('Tam2_unwantedCols.pkl', 'rb') as f:
        unwantedColumns = pickle.load(f)
        
    testX = testX.drop(unwantedColumns, axis = 1)
    return testX

In [18]:
# dataSet['SEO description']

In [19]:
train = getTrainX(xTrain, yTrain)

trainModel(train, yTrain)
trainBkp['output'] = predict(train)

test = getTestX(xTest)
testBkp['output'] = predict(test)


[0, 1]


In [20]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

def generateMetrics(actualY, predictedY):
    accuracy = accuracy_score(actualY, predictedY)
    precision = precision_score(actualY, predictedY, average='weighted') 
    recall = recall_score(actualY, predictedY, average='weighted') 
    f1Score = f1_score(actualY, predictedY, average='weighted') 
    
    return accuracy, precision, recall, f1Score

In [21]:

models = ['LogisticRegression']

outputMetrics = pd.DataFrame(columns=['Model', 'TrainAccuracy', 'TrainPrecision', 'TrainRecall', 'TrainF1Score', 'TestAccuracy', 'TestPrecision', 'TestRecall', 'TestF1Score'])

for model in models:
    trainAccuracy, trainPrecision, trainRecall, trainF1Score = generateMetrics( yTrain, trainBkp['output'])
    testAccuracy, testPrecision, testRecall, testF1Score  = generateMetrics(yTest, testBkp['output'])
    modelMetrics = pd.DataFrame({'Model': model, 
                                  'TrainAccuracy': trainAccuracy, 'TestAccuracy': testAccuracy,
                                  'TrainPrecision': trainPrecision, 'TestPrecision': testPrecision, 
                                  'TrainRecall': trainRecall,  'TestRecall': testRecall, 
                                  'TrainF1Score': trainF1Score, 'TestF1Score': testF1Score}, 
                                 index=[0])
    
    outputMetrics = pd.concat([outputMetrics, modelMetrics], ignore_index = True)


In [22]:
outputMetrics

,Model,TrainAccuracy,TrainPrecision,TrainRecall,TrainF1Score,TestAccuracy,TestPrecision,TestRecall,TestF1Score
0,LogisticRegression,0.840026,0.841268,0.840026,0.840201,0.681579,0.681392,0.681579,0.681254


In [23]:
predDS = pd.read_excel('/Users/amulya/Desktop/newyork1.xlsx')
# //downloaded
xPred = predDS[['Company', 'Lists', 'Industry', 'Keywords', 'SEO Description',
       'Technologies', 'Short Description']]
# predDS.head()

FileNotFoundError: [Errno 2] No such file or directory: '/Users/amulya/Desktop/newyork1.xlsx'

In [ ]:
predBkp = predDS.copy()

trainBkp['output'] = predict(train)

predX = getTestX(xPred)
predBkp['output'] = predict(predX)


In [ ]:
predBkp['output'] = predBkp['output'].replace({1: 'ICP', 0 : 'Not an ICP'})
predBkp.to_excel('/Users/amulya/Desktop/newyork_output.xlsx',index=False)
